# FAISS

>[Facebook AI Similarity Search (Faiss)](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)は、密なベクトルの効率的な類似性検索とクラスタリングのためのライブラリです。RAMに収まらない可能性のある任意のサイズのベクトルセットで検索するアルゴリズムを含んでいます。また、評価とパラメータチューニングのための補助コードも含まれています。

[Faissのドキュメンテーション](https://faiss.ai/)。

このノートブックでは、`FAISS`ベクトルデータベースに関連する機能の使用方法を示します。

In [6]:
#!pip install faiss
# OR
!pip install faiss-cpu
!pip install tiktoken

  Using cached tiktoken-0.4.0-cp311-cp311-win_amd64.whl (635 kB)
  Using cached regex-2023.6.3-cp311-cp311-win_amd64.whl (268 kB)


OpenAIEmbeddingsを使用したいので、OpenAI APIキーを取得する必要があります。

In [7]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# AVX2最適化なしでFAISSを初期化する必要がある場合は、以下の行のコメントを解除してください
# os.environ['FAISS_NO_AVX2'] = '1'

In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [9]:
from langchain.document_loaders import TextLoader

loader = TextLoader("../../../state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [10]:
db = FAISS.from_documents(docs, embeddings)

query = "ケタンジ・ブラウン・ジャクソンについて、大統領は何と言いましたか？"
docs = db.similarity_search(query)

In [11]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## スコア付き類似性検索
FAISS特有のメソッドがいくつかあります。その一つが`similarity_search_with_score`で、これにより文書だけでなく、クエリとそれらの文書との間の距離スコアも返すことができます。返される距離スコアはL2距離です。したがって、スコアが低いほど良いです。

In [12]:
docs_and_scores = db.similarity_search_with_score(query)

In [13]:
docs_and_scores[0]

(Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../../state_of_the_union.txt'}),
 0.45091543)

また、`similarity_search_by_vector`を使用して、与えられた埋め込みベクトルに類似した文書を検索することも可能です。これは文字列の代わりに埋め込みベクトルをパラメータとして受け取ります。

In [14]:
embedding_vector = embeddings.embed_query(query)
docs_and_scores = db.similarity_search_by_vector(embedding_vector)

## 保存と読み込み
FAISSインデックスも保存と読み込みが可能です。これは、毎回インデックスを再作成する必要がないため便利です。

In [15]:
db.save_local("faiss_index")

In [16]:
new_db = FAISS.load_local("faiss_index", embeddings)

In [17]:
docs = new_db.similarity_search(query)

In [18]:
docs[0]

Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../../state_of_the_union.txt'})

## マージ
また、2つのFAISSベクトルストアをマージすることも可能です。

In [19]:
db1 = FAISS.from_texts(["foo"], embeddings)
db2 = FAISS.from_texts(["bar"], embeddings)

In [20]:
db1.docstore._dict

{'8e96232a-92bd-485c-aefb-e86aba99c025': Document(page_content='foo', metadata={})}

In [21]:
db2.docstore._dict

{'7f18b347-38a3-4c3f-b50b-3df0ab3a59e5': Document(page_content='bar', metadata={})}

In [22]:
db1.merge_from(db2)

In [23]:
db1.docstore._dict

{'8e96232a-92bd-485c-aefb-e86aba99c025': Document(page_content='foo', metadata={}),
 '7f18b347-38a3-4c3f-b50b-3df0ab3a59e5': Document(page_content='bar', metadata={})}

## フィルタリング付き類似性検索
FAISSベクトルストアはフィルタリングもサポートしていますが、FAISSはネイティブではフィルタリングをサポートしていないため、手動で行う必要があります。これは、まず`k`より多くの結果を取得し、それからフィルタリングすることで行われます。メタデータに基づいて文書をフィルタリングすることができます。また、任意の検索メソッドを呼び出す際に`fetch_k`パラメータを設定して、フィルタリング前に取得したい文書の数を設定することもできます。以下に簡単な例を示します：

In [24]:
from langchain.schema import Document

list_of_documents = [
    Document(page_content="foo", metadata=dict(page=1)),
    Document(page_content="bar", metadata=dict(page=1)),
    Document(page_content="foo", metadata=dict(page=2)),
    Document(page_content="barbar", metadata=dict(page=2)),
    Document(page_content="foo", metadata=dict(page=3)),
    Document(page_content="bar burr", metadata=dict(page=3)),
    Document(page_content="foo", metadata=dict(page=4)),
    Document(page_content="bar bruh", metadata=dict(page=4)),
]
db = FAISS.from_documents(list_of_documents, embeddings)
results_with_scores = db.similarity_search_with_score("foo")
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 1}, Score: 5.1599612373143776e-15
Content: foo, Metadata: {'page': 2}, Score: 5.1599612373143776e-15
Content: foo, Metadata: {'page': 3}, Score: 5.1599612373143776e-15
Content: foo, Metadata: {'page': 4}, Score: 5.1599612373143776e-15


次に、同じクエリを呼び出しますが、`page = 1`のみをフィルタリングします。

In [25]:
results_with_scores = db.similarity_search_with_score("foo", filter=dict(page=1))
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 1}, Score: 5.1599612373143776e-15
Content: bar, Metadata: {'page': 1}, Score: 0.3131446838378906


同様のことは`max_marginal_relevance_search`でも行うことができます。

In [26]:
results = db.max_marginal_relevance_search("foo", filter=dict(page=1))
for doc in results:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}")

Content: foo, Metadata: {'page': 1}
Content: bar, Metadata: {'page': 1}


以下は、`similarity_search`を呼び出す際に`fetch_k`パラメータを設定する方法の例です。通常、`fetch_k`パラメータは`k`パラメータよりも大きく設定します。これは、`fetch_k`パラメータがフィルタリング前に取得される文書の数であるためです。`fetch_k`を低い数値に設定すると、フィルタリングするための十分な文書が得られない可能性があります。

In [27]:
results = db.similarity_search("foo", filter=dict(page=1), k=1, fetch_k=4)
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 1}, Score: 5.1599612373143776e-15
Content: bar, Metadata: {'page': 1}, Score: 0.3131446838378906
